<a href="https://colab.research.google.com/github/svishakan/Artificial-Intelligence/blob/master/AI%20-%20Lab%2006%20-%20DPLL%20Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Intelligence — Lab — Exercise 06

#### 6 October 2020

## Davis–Putnam–Logemann–Loveland (DPLL) algorithm

Implement David-Putnam algorithm for checking the satisfiability of a sentence represented in propositional logic.
<hr>

The Davis–Putnam–Logemann–Loveland (DPLL) algorithm is a complete, 
backtracking-based search algorithm for deciding the satisfiability of propositional logic formulae 
in conjunctive normal form, i.e. for solving the CNF-SAT problem.
<hr>

Theory is adapted from Artificial Intelligence: A Modern Approach by Peter Norvig & Stuart J. Russell

<hr>
This algorithm is very similar to Backtracking-Search. It recursively enumerates possible models in a depth-first fashion with the following improvements over algorithms like Truth Table Entailment.

1. Early termination:
In certain cases, the algorithm can detect the truth value of a statement using just a partially completed model. For example, $(P\lor Q)\land(P\lor R)$ is true if P is true, regardless of other variables. This reduces the search space significantly.
<br><br>
2. Pure symbol heuristic:
A symbol that has the same sign (positive or negative) in all clauses is called a pure symbol. It isn't difficult to see that any satisfiable model will have the pure symbols assigned such that its parent clause becomes true. For example, $(P\lor\neg Q)\land(\neg Q\lor\neg R)\land(R\lor P)$ has P and Q as pure symbols and for the sentence to be true, P has to be true and Q has to be false. The pure symbol heuristic thus simplifies the problem a bit.
<br><br>
3. Unit clause heuristic:
In the context of DPLL, clauses with just one literal and clauses with all but one false literals are called unit clauses. If a clause is a unit clause, it can only be satisfied by assigning the necessary value to make the last literal true. We have no other choice.
<br><br>
Assigning one unit clause can create another unit clause. For example, when P is false, $(P\lor Q)$ becomes a unit clause, causing true to be assigned to Q. A series of forced assignments derived from previous unit clauses is called unit propagation. In this way, this heuristic simplifies the problem further.
<hr>


In [11]:
class DPLL:
    true_literals = set()
    false_literals = set()
    pure_literals = set()
    cnf = []
    literals = []
    unit_props, num_pure, branches = 0, 0, 0
    
    def __init__(self, formula):
        """To initialize the DPLL Solver with the required CNF Form & Literals. """
        
        self.cnf = formula.split(", ")
        self.literals = self.find_literals(self.cnf)
        self.pure_literals = set()

        print("Initial Formula:\t\t\t", self)
        
        
    def __str__(self):
        """To convert the Pythonic-list used for CNF notation into a readable formula in CNF. """
        
        cnf_str = ""
        
        for clause in self.cnf:  #represent each claue in CNF
            if len(clause) > 0:
                cnf_str += '(' + clause.replace(' ', ' ∨ ') + ') ∧ '
    
        if cnf_str == "":       #formula is empty
            cnf_str = "()"
            
        if cnf_str[-2] == "∧":  #removing the final '∧' added which is un-wanted.
            cnf_str = cnf_str[:-2:] 
    
        return cnf_str

    
    def find_literals(self, formula):
        """Finds the literals existing in a given formula. """

        literals = [literal for literal in list(set(''.join(formula))) if literal.isalpha()]

        return literals
    
    def remove_pure_literals(self):
        """Finds the pure literals in the given formula. """

        cnf = list(set(self.cnf))
        unit_clauses = [clause for clause in cnf if len(clause) < 3]
        unit_clauses = list(set(unit_clauses))
        current_literals = self.find_literals(cnf)  #Find literals in the current formula

        for literal in current_literals:  #Check for literals like X
            is_pure_literal = False

            for i in range(len(cnf)):
                if '¬' + literal not in cnf[i]: 
                    is_pure_literal = True
                else:  #If ¬X exists in other clauses
                    is_pure_literal = False
                    break
            
            if is_pure_literal:  #If X is a pure literal
                self.num_pure += 1
                self.pure_literals.add(literal)
                self.true_literals.add(literal)  #Add X to True literals
                i = 0

                while True:  #Remove clauses containing X
                    if i >= len(cnf):
                        break

                    if literal in cnf[i]:
                        cnf.remove(cnf[i])
                    
                        i -= 1
                    i += 1

        for literal in current_literals:  #Check for literals like ¬X 
            literal = '¬' + literal  #Search for literals of the type ¬X
            is_pure_literal = False

            for i in range(len(cnf)):
                if literal in cnf[i]: 
                    is_pure_literal = True
                else:  #If X exists in other clauses
                    is_pure_literal = False
                    break
            
            if is_pure_literal:  #If ¬X is a pure literal
                self.num_pure += 1
                self.pure_literals.add(literal)
                self.false_literals.add(literal[-1])  #Add X to False literals
                i = 0

                while True:  #Remove clauses containing ¬X
                    if i >= len(cnf):
                        break

                    if literal in cnf[i]:
                        cnf.remove(cnf[i])
                    
                        i -= 1
                    i += 1
        
        self.cnf = cnf

        print("Pure Literals:\t\t\t\t", list(self.pure_literals))
        print("CNF after removing Pure Literals:\t", self)

        return cnf

    
    def unit_clause_propagation(self):
        """Peform the unit-clause propagation on the given formula. """
        
        new_true_literals  =  []
        new_false_literals =  []
        
        print("\nCurrent Formula:\t\t\t", self)
        
        cnf = list(set(self.cnf))
        unit_clauses = [clause for clause in cnf if len(clause) < 3]   #clause of len < 3 is a unit clause
        unit_clauses = list(set(unit_clauses))
        
        if unit_clauses:  #Assign truth values to unit clauses for satisfying the given formula
            for unit in unit_clauses:
                self.unit_props += 1
                
                if '¬' in unit:  #False unit clause
                    self.false_literals.add(unit[-1])
                    new_false_literals.append(unit[-1])
                    i = 0
                    
                    while True:
                        if unit in cnf[i]:  #Remove the unit clause from the formula
                            cnf.remove(cnf[i])
                            i -= 1
                        
                        elif unit[-1] in cnf[i]:  #If a ¬(unit clause) exists in the formula
                            cnf[i] = cnf[i].replace(unit[-1], '').strip()
                    
                        i += 1
                    
                        if i >= len(cnf):
                            break
        
                else:  #True unit clause
                    self.true_literals.add(unit)
                    new_true_literals.append(unit)
                    i = 0
                
                    while True:
                        if '¬' + unit in cnf[i]:  #If a ¬(unit clause) exists in the formula
                            cnf[i] = cnf[i].replace('¬' + unit, '').strip()
                        
                            if '  ' in cnf[i]:  #Empty clause
                                cnf[i] = cnf[i].replace('  ', ' ')
                        
                        elif unit in cnf[i]:  #Remove the unit clause from the formula
                            cnf.remove(cnf[i])
                            i -= 1
                            
                        i += 1
                        
                        if i >= len(cnf):
                            break
        
        self.cnf = cnf

        print("Current Unit Clauses:\t\t\t", unit_clauses)
        print("CNF after doing Unit Propagation:\t", self)

        return cnf, new_true_literals, new_false_literals
    
    
    def DPLL_Algorithm(self):
        """Perform the David-Putnam-Logemann-Loveland algorithm on the given formula. """
        
        self.true_literals = set(self.true_literals)
        self.false_literals = set(self.false_literals)
        
        self.branches += 1  #Since this is a new branch

        self.cnf, new_true_literals, new_false_literals = self.unit_clause_propagation()  #Do unit-clause propagation

        if sum(len(clause) == 0 for clause in self.cnf):  #If there are empty parantheses, indicating contradiction
            
            for literal in new_true_literals:   #clear every newly set true literal
                self.true_literals.remove(literal)
            
            for literal in new_false_literals:  #clear every newly set false literal
                self.false_literals.remove(literal)
                
            print("\nNull clause found. Backtracking...")
            
            return False  #Backtrack, since we cannot simplify anymore, and it is unsatisfiable in the current branch


        self.cnf = self.remove_pure_literals()  #Removing pure literal containing clauses from the formula

        cnf = self.cnf

        if len(self.cnf) == 0:  #No more clauses to satisfy
            return True

        
        self.literals = self.find_literals(self.cnf)
        #form the literals again from the updated formula
        
        first_literal = self.literals[0]  #choose the first literal to set as True/False to find satisfying assignments
        
        self.cnf = cnf + [first_literal]  #Try with first literal set to True
        print("\nTrying with {0} as True...".format(first_literal))
        
        if self.DPLL_Algorithm():  #recursively work on the new formula
            return True
        
        self.cnf = cnf + ['¬' + first_literal]  #Try with first literal set to False (i.e ¬(first literal) is True)
        print("\nTrying with ¬{0} as True...".format(first_literal))
        
        if self.DPLL_Algorithm():  #recursively work on the new formula
            return True

        
        else:  #no satisfying assignments were found to the literals of the formula
            self.cnf = cnf
            
            for literal in new_true_literals:   #remove all the assignments
                self.true_literals.remove(literal)
            
            for literal in new_false_literals:  #remove all the assignments
                self.false_literals.remove(literal)
            
            return False  #the formula is unsatisfiable.
        
    
    def print_result(self, satisfiability):
        """Print the result of the DPLL Algorithm on the formula along with its literals' assignments and statistics. """
        
        if satisfiability == True:
            print("\nThe given CNF statement is satisfiable.")
            print("\nSolution: ")
            
            for literal in self.true_literals:
                print("\t"+literal, "= True")
                
            for literal in self.false_literals:
                print("\t"+literal, "= False")
            
        else:
            print("\nThe given CNF statement was unsatisfiable.")
            
        print("\nNo. of branches:\t\t", self.branches)
        print("\nNo. of unit propagations:\t", self.unit_props)
        print("\nNo. of pure literals removed:\t", self.num_pure)


<hr>
Running DPLL Algorithm for $(X \lor Y \lor Z)\land(X \lor\neg Y \lor Z)\land(\neg X \lor Y \lor\neg Z)\land(\neg Z)$
<hr>

In [16]:
dpll_solver = DPLL("X Y Z, X ¬Y Z, ¬X Y ¬Z, ¬Z")
satisfiability = dpll_solver.DPLL_Algorithm()
dpll_solver.print_result(satisfiability)

Initial Formula:			 (X ∨ Y ∨ Z) ∧ (X ∨ ¬Y ∨ Z) ∧ (¬X ∨ Y ∨ ¬Z) ∧ (¬Z) 

Current Formula:			 (X ∨ Y ∨ Z) ∧ (X ∨ ¬Y ∨ Z) ∧ (¬X ∨ Y ∨ ¬Z) ∧ (¬Z) 
Current Unit Clauses:			 ['¬Z']
CNF after doing Unit Propagation:	 (X ∨ ¬Y) ∧ (X ∨ Y) 
Pure Literals:				 ['X']
CNF after removing Pure Literals:	 ()

The given CNF statement is satisfiable.

Solution: 
	X = True
	Z = False

No. of branches:		 1

No. of unit propagations:	 1

No. of pure literals removed:	 1


<hr>
Running DPLL Algorithm for $(\neg P \lor Q)\land(\neg Q \lor\neg R)\land(\neg S \lor P)\land(P \lor Q \lor\neg R)$
<hr>

In [13]:
dpll_solver = DPLL("¬P Q, ¬Q ¬R, R ¬S, ¬S P, P Q ¬R")
satisfiability = dpll_solver.DPLL_Algorithm()
dpll_solver.print_result(satisfiability)

Initial Formula:			 (¬P ∨ Q) ∧ (¬Q ∨ ¬R) ∧ (R ∨ ¬S) ∧ (¬S ∨ P) ∧ (P ∨ Q ∨ ¬R) 

Current Formula:			 (¬P ∨ Q) ∧ (¬Q ∨ ¬R) ∧ (R ∨ ¬S) ∧ (¬S ∨ P) ∧ (P ∨ Q ∨ ¬R) 
Current Unit Clauses:			 []
CNF after doing Unit Propagation:	 (¬Q ∨ ¬R) ∧ (R ∨ ¬S) ∧ (¬S ∨ P) ∧ (¬P ∨ Q) ∧ (P ∨ Q ∨ ¬R) 
Pure Literals:				 []
CNF after removing Pure Literals:	 (¬Q ∨ ¬R) ∧ (R ∨ ¬S) ∧ (¬P ∨ Q) ∧ (¬S ∨ P) ∧ (P ∨ Q ∨ ¬R) 

Trying with S as True...

Current Formula:			 (¬Q ∨ ¬R) ∧ (R ∨ ¬S) ∧ (¬P ∨ Q) ∧ (¬S ∨ P) ∧ (P ∨ Q ∨ ¬R) ∧ (S) 
Current Unit Clauses:			 ['S']
CNF after doing Unit Propagation:	 (¬Q ∨ ¬R) ∧ (R) ∧ (P) ∧ (¬P ∨ Q) ∧ (P ∨ Q ∨ ¬R) 
Pure Literals:				 []
CNF after removing Pure Literals:	 (¬Q ∨ ¬R) ∧ (P) ∧ (¬P ∨ Q) ∧ (R) ∧ (P ∨ Q ∨ ¬R) 

Trying with P as True...

Current Formula:			 (¬Q ∨ ¬R) ∧ (P) ∧ (¬P ∨ Q) ∧ (R) ∧ (P ∨ Q ∨ ¬R) ∧ (P) 
Current Unit Clauses:			 ['R', 'P']
CNF after doing Unit Propagation:	 (¬Q) ∧ (Q) 
Pure Literals:				 []
CNF after removing Pure Literals:	 (Q) ∧ (¬Q) 

Trying with Q a

<hr>
Custom formula : Separate clauses in formula by commas ", " and literals inside a clause with spaces. Use "¬" for negation of a literal.
<hr>

In [14]:
dpll_solver = DPLL(input("Enter a formula in CNF: "))
satisfiability = dpll_solver.DPLL_Algorithm()
dpll_solver.print_result(satisfiability)

Enter a formula in CNF: P, ¬P
Initial Formula:			 (P) ∧ (¬P) 

Current Formula:			 (P) ∧ (¬P) 
Current Unit Clauses:			 ['P', '¬P']
CNF after doing Unit Propagation:	 ()

Null clause found. Backtracking...

The given CNF statement was unsatisfiable.

No. of branches:		 1

No. of unit propagations:	 2

No. of pure literals removed:	 0


In [15]:
dpll_solver = DPLL("X Y, Y ¬X, ¬X ¬Y")
satisfiability = dpll_solver.DPLL_Algorithm()
dpll_solver.print_result(satisfiability)

Initial Formula:			 (X ∨ Y) ∧ (Y ∨ ¬X) ∧ (¬X ∨ ¬Y) 

Current Formula:			 (X ∨ Y) ∧ (Y ∨ ¬X) ∧ (¬X ∨ ¬Y) 
Current Unit Clauses:			 []
CNF after doing Unit Propagation:	 (Y ∨ ¬X) ∧ (¬X ∨ ¬Y) ∧ (X ∨ Y) 
Pure Literals:				 []
CNF after removing Pure Literals:	 (Y ∨ ¬X) ∧ (¬X ∨ ¬Y) ∧ (X ∨ Y) 

Trying with Y as True...

Current Formula:			 (Y ∨ ¬X) ∧ (¬X ∨ ¬Y) ∧ (X ∨ Y) ∧ (Y) 
Current Unit Clauses:			 ['Y']
CNF after doing Unit Propagation:	 (¬X) 
Pure Literals:				 ['¬X']
CNF after removing Pure Literals:	 ()

The given CNF statement is satisfiable.

Solution: 
	Y = True
	X = False

No. of branches:		 2

No. of unit propagations:	 1

No. of pure literals removed:	 1
